In [35]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [36]:
import os

In [37]:
%pwd

'e:\\projects_final file\\Text  Summarization\\Text-Summarization-Project'

To move whole work in main directory

In [38]:
os.chdir('../')

In [39]:
%pwd

'e:\\projects_final file\\Text  Summarization'

## 1.Update config.yaml file:


artifacts_root: artifacts

#data dowloading and making path for saving it

data_ingestion:

  root_dir:artifacts/data_ingestion

  source_URL:https://github.com/ShalinVachheta017/Text-Summarization-Project/raw/main/summarizer-data.zip 

  local_data_file:artifacts/data_ingestion/data.zip
  
  unzip_dir = artifacts/data_ingestion/


## 2. Update params.yaml file ..skip for now

## 3. Update entity:

In [40]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
#Make a class for data ingestion configuration with file and its types
class DataIngestionConfig:
    root_dir: Path #it/root_dir in config.yaml is a path type so Path will be here
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## 4.Update configuration manager in src config

In [41]:
from textSummarizer.constan import * #to import necessary constants
from textSummarizer.utils.common import read_yaml, create_directories #to read yaml file and create directories

The ConfigurationManager class is responsible for reading configuration and parameter files, creating necessary directories, and providing configuration details for data ingestion. 

It initializes with file paths for configuration and parameters, reads these files, and sets up directories as specified in the configuration.

In [55]:
import yaml
from box import ConfigBox
from box.exceptions import BoxValueError
from pathlib import Path

def read_yaml(PARAMS_FILE_PATH: Path,CONFIG_FILE_PATH: Path) -> ConfigBox:
    try:
        with open(PARAMS_FILE_PATH, 'r') as yaml_file:
            content = yaml.safe_load(yaml_file)
            if not content:
                raise ValueError("YAML file is empty")
            return ConfigBox(content)
    except BoxValueError as e:
        raise ValueError(f"Error in converting YAML to ConfigBox: {e}")
    except Exception as e:
        raise e


In [56]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

## 5.Update the components

In [57]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [58]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    # download_file: Downloads the data file from the source URL to the local directory
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(
                f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
            
    #extract_zip_file: Extracts the zip file into the data directory
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [59]:
try:
    config = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/config.yaml'